In [3]:
from google.cloud import aiplatform
from google.cloud.aiplatform_v1.types import SampledShapleyAttribution
from google.cloud.aiplatform_v1.types.explanation import ExplanationParameters

project_id = "dataanalytics-347914"
region = "us-central1"
bucket = "gs://udemy-gcp-mlops"

aiplatform.init(project=project_id, location=region, staging_bucket=bucket)

<h4>Custom Model Training

In [4]:
job = aiplatform.CustomTrainingJob(
    display_name="bikeshare-training-job",
    script_path="model-training-code.py",
    container_uri="us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest",
    requirements=["gcsfs"]
)

job.run(
    replica_count=1,
    machine_type="n1-standard-4",
    sync=True
)

job.wait()

Training script copied to:
gs://udemy-gcp-mlops/aiplatform-2024-09-14-19:47:05.723-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://udemy-gcp-mlops/aiplatform-custom-training-2024-09-14-19:47:06.368 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7979175777058947072?project=936546808722
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8284452981487697920?project=936546808722
CustomTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/7979175777058947072 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/7979175777058947072 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/936546808722/locations/us-central1/trainingPipelines/7979175777058947072 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/936546808722/lo

<h4>Upload Trained Model to Model Registry 

In [ ]:
display_name = "bikeshare-model-sdk"
artifact_uri = "gs://udemy-gcp-mlops/bikeshare-model/artifact/"
serving_container_image_uri = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"

exp_metadata = {"inputs": {"input_features": {}}, "outputs": {"predicted_outcome": {}}}

model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        explanation_metadata=exp_metadata,
        explanation_parameters=ExplanationParameters(
                sampled_shapley_attribution=SampledShapleyAttribution(path_count=25)
            ),
        sync=False
    )

model.wait()

Creating Model
Create Model backing LRO: projects/936546808722/locations/us-central1/models/8291611798616932352/operations/3314793533566615552
Model created. Resource name: projects/936546808722/locations/us-central1/models/8291611798616932352@1
To use this Model in another session:
model = aiplatform.Model('projects/936546808722/locations/us-central1/models/8291611798616932352@1')


<h4>Deploy Model to Vertex Endpoint

In [6]:
deployed_model_display_name = "bikeshare-endpoint-exp-v1"
traffic_split = {"0": 100}
machine_type = "n1-standard-4"
min_replica_count = 1
max_replica_count = 1

endpoint = model.deploy(
        deployed_model_display_name=deployed_model_display_name,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count
    )

Creating Endpoint
Create Endpoint backing LRO: projects/936546808722/locations/us-central1/endpoints/6725916036756930560/operations/2070111186552094720
Endpoint created. Resource name: projects/936546808722/locations/us-central1/endpoints/6725916036756930560
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/936546808722/locations/us-central1/endpoints/6725916036756930560')
Deploying model to Endpoint : projects/936546808722/locations/us-central1/endpoints/6725916036756930560
Deploy Endpoint model backing LRO: projects/936546808722/locations/us-central1/endpoints/6725916036756930560/operations/5175343129624051712
Endpoint model deployed. Resource name: projects/936546808722/locations/us-central1/endpoints/6725916036756930560


In [7]:
endpoint = aiplatform.Endpoint('projects/936546808722/locations/us-central1/endpoints/6725916036756930560')

<h4> Run Predictions

In [8]:
instances_list = [
    [0.24, 0.81, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0.8,0.27, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,0.0,0.0,0.0, 0.0, 0.0, 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0]
    ]

prediction = endpoint.predict(instances_list)
print(prediction)

Prediction(predictions=[4.122591586485379, 5.902414992487585], deployed_model_id='7711755754406936576', metadata=None, model_version_id='1', model_resource_name='projects/936546808722/locations/us-central1/models/8291611798616932352', explanations=None)


<h4> Define the input features used for model training

In [9]:
features = ['time',
 'coupon',
 'expiration',
 'age',
 'education',
 'occupation',
 'income',
 'Bar',
 'CoffeeHouse',
 'CarryAway',
 'Restaurant20To50',
 'toCoupon_GEQ15min',
 'toCoupon_GEQ25min',
 'direction_same',
 'passanger_destination',
 'marital_hasChildren',
 'temperature_weather']

In [10]:
response = endpoint.explain(instances=instances_list)

for explanation in response.explanations:
    attributions = explanation.attributions
    for attribution in attributions:
            print("  attribution")
            print("   baseline_output_value:", attribution.baseline_output_value)
            print("   instance_output_value:", attribution.instance_output_value)
            print("   output_display_name:", attribution.output_display_name)
            print("   approximation_error:", attribution.approximation_error)
            print("   output_name:", attribution.output_name)
            output_index = attribution.output_index
            for output_index in output_index:
                # print("   output_index:", output_index)
                attrs = attribution.feature_attributions
                rows = {"feature_name": [], "attribution": []}
                for i, val in enumerate(features):
                    rows["feature_name"].append(val)
                    rows["attribution"].append(attrs["input_features"][i])
            
            print(rows)
            print("-------------------------------------")

  attribution
   baseline_output_value: 4.1130071035862565
   instance_output_value: 4.122591586485379
   output_display_name: 
   approximation_error: 0.0007459996783010174
   output_name: predicted_outcome
{'feature_name': ['time', 'coupon', 'expiration', 'age', 'education', 'occupation', 'income', 'Bar', 'CoffeeHouse', 'CarryAway', 'Restaurant20To50', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min', 'direction_same', 'passanger_destination', 'marital_hasChildren', 'temperature_weather'], 'attribution': [0.3428701193940526, -0.2371793638651169, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}
-------------------------------------
  attribution
   baseline_output_value: 4.1130071035862565
   instance_output_value: 5.902414992487585
   output_display_name: 
   approximation_error: 0.00010089341089507533
   output_name: predicted_outcome
{'feature_name': ['time', 'coupon', 'expiration', 'age', 'education', 'occupation', 'income', 'Bar', 'CoffeeHouse', 'CarryAway', 'R

In [11]:
# model = aiplatform.Model('projects/1090925531874/locations/us-central1/models/6594037313586593792')

gcs_input_uri = 'gs://udemy-gcp-mlops/bikeshare-model/batch.csv'
BUCKET_URI = "gs://udemy-gcp-mlops/bikeshare-model/bikeshare-batch-prediction-output"

batch_predict_job = model.batch_predict(
    job_display_name="bikeshare_batch_predict",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    generate_explanation=True,
    sync=False
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/936546808722/locations/us-central1/batchPredictionJobs/5756297517230718976
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/936546808722/locations/us-central1/batchPredictionJobs/5756297517230718976')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5756297517230718976?project=936546808722
BatchPredictionJob projects/936546808722/locations/us-central1/batchPredictionJobs/5756297517230718976 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/936546808722/locations/us-central1/batchPredictionJobs/5756297517230718976 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/936546808722/locations/us-central1/batchPredictionJobs/5756297517230718976 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/936546808722/locations/us-central1/batchPredictionJobs/